In [1]:
import os
from datetime import datetime
from pathlib import Path

In [2]:
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [3]:
from bs4 import BeautifulSoup
import lxml

In [4]:
from django.contrib.auth.models import User

In [5]:
from taxonomy.models import Species
from photos.models import Photo

In [35]:
bucket_base_url = "https://bucket-9tbcqs.s3.ap-northeast-2.amazonaws.com/"
path = Path("../assets/images/crop")

def find_file(path):
    for new_path in path.glob("*"): 
        if new_path.is_dir():
            return find_file(new_path)
        else:
            return new_path

for filename in path.glob("*"):
    f = find_file(filename) 
    pid = f.parent.name
    image_path = bucket_base_url + f"images/{datetime.now().strftime('%Y%m')}/{pid}_00001{f.suffix}"
    thumbnail_path = bucket_base_url + f"thumbnails/thumbnail_{pid}{f.suffix}"
    obj = Species.objects.get(pid=pid)
    Photo.objects.create(user = User.objects.get(username__exact="관리자"), 
                         plant = obj,
                         url = image_path,
                         thumbnail = thumbnail_path,
                         is_published=True,
                         is_peer_reviewed=True)

In [6]:
#base_url = "https://bucket-9tbcqs.s3.ap-northeast-2.amazonaws.com/"
base_url = "https://d3gjf5t0bpg9ql.cloudfront.net/"
dest_url = "https://d71pexumifcjq.cloudfront.net/"
queryset = Photo.objects.all()
for query in queryset:
    thumbnail_replaced = query.thumbnail.replace(base_url, dest_url)
    image_replaced = query.url.replace(base_url, dest_url)
    query.url = image_replaced
    query.thumbnail = thumbnail_replaced
    query.save()